In [4]:
from PIL import Image, ImageDraw
import numpy as np
import random
from threading import Thread
import os

import subprocess, sys, base64, imutils, cv2
import time


In [5]:
Name_Good = 'Good_desc.dat'
Name_Bad = 'Bad_desc.dat'

Folder_Good = 'Good'
Folder_Bad = 'Bad'
black_name = '.DS_Store'

In [4]:
image = Image.open('2.jpeg')
a = [int(i/10) for i in image.size]
test_image = image.resize(a, Image.ANTIALIAS)
name_new_file = 'vou.png'
test_image.save(name_new_file, optimize = True, quality = 95)

#foo.save("path\\to\\save\\image_scaled_opt.jpg",optimize=True,quality=95)

FileNotFoundError: [Errno 2] No such file or directory: '2.jpeg'

In [3]:
image = Image.open(name_new_file)
path_save = 'Good/'

draw = ImageDraw.Draw(image) 
width = image.size[0]
height = image.size[1] 
pix = image.load()

step = 1
size = int(max(height, width)* 2**0.5) + 1
last_size = [width, height]

color_delete = [[255] * 4, [150] * 4]



NameError: name 'name_new_file' is not defined

In [25]:
def simple_rotate_value(i, j, phi):
    phi = phi * np.pi / 180
    x = i * np.cos(phi) + j * np.sin(phi)
    y = -i * np.sin(phi) + j * np.cos(phi)
    return int(x), int(y)


def get_noize():
    return tuple([random.randint(0, 255) for i in range(4)])

def create_noise(pix, i, j, color = None):
    
    error = 0
    color = color if color else 100
    flag = full_delete(pix[i,j], [color]*4)
    
    if flag:
        return get_noize(), flag
    else:
        return pix[i, j], flag
    
    
def full_delete(pix, color, error = 0):
    
    f_error = lambda x, color: np.mean([i - j for i,j in zip(x[:-1], color)])    
    return f_error(pix, color) > error

def delete_color(pix, color_list, error):
    # удаление цвета, больше заданного
    
    if color_list:
        if type(color_list) != list:
            color_list = [[color_list] * 4]
        elif type(color_list[0]) != list:
            color_list = [color_list]
    else:
        color_list = [[255] * 4]

    f_error = lambda x, color = [0, 0, 0, 0]: np.mean([abs(i - j) for i,j in zip(x[:-1], color)])    
    
    flag_error = False
    
    for i in color_list:
        flag_error = abs(f_error(pix, i)) > error
        
        if flag_error:
            break
        
    #print(pix, f_error(pix), color, flag_error)
    return flag_error


def rotate_image(image, noise_image, pix, list_point, path_save, phi):
    # поворот картинки на заданный градус
    
    new_image = noise_image.copy()
    draw = ImageDraw.Draw(new_image)
    
    for i in list_point:
        x, y = simple_rotate_value(i[0] - new_image.size[0]/2, i[1] - new_image.size[1]/2, phi)
        draw.point((size/2 + x, size/2 + y), pix[i[0], i[1]])
        
    new_image.save(path_save)
    
def create_noise_image(path_save = None, image = None, size = None):
    # создание шумной картинки заданного размера
    if image:
        new_image = image.copy()
    else:
        new_image = Image.new('RGBA', (size, size))
    
    size = new_image.size[0]
    draw = ImageDraw.Draw(new_image)
    
    for i in range(size):
        for j in range(size):
            draw.point((i, j), get_noize())
    
    if path_save:
        new_image.save(path_save)
        
    return new_image

def change_image(pix, size, last_size, path = None, color_delete = None):
    # зашумление картинки
    #("path:", path)
    
    new_image = Image.new('RGBA', (size, size))
    new_draw = ImageDraw.Draw(new_image)
    
    bias = [(size - i)/2 for i in last_size]
    list_point = []
    
    for i in range(size):
        for j in range(size):
            if abs(i - size/2) < last_size[0]/2 and abs(j - size/2) < last_size[1]/2:
                new_pix, flag = create_noise(pix, i - bias[0], j - bias[1], color = color_delete)
                new_draw.point((i, j), new_pix)
                
                if not flag:
                    list_point.append((i, j))
                
            else:
                new_draw.point((i, j), get_noize())
            
    if path:
        new_image.save(path)

    return new_image, list_point
    
def create_name(size, number = 1, bias = ' '):
    ret = [number, 0, 0, size, size]
    ret_str = bias
    for i in ret:
        ret_str += str(i) + ' '
    return ret_str[:-1]


def change_value(if_value, else_value, check, flag_work = True):
    if flag_work:
        ret = if_value if if_value == check else else_value
    else:
        ret = if_value if if_value != check else else_value
    return ret

def change_str_to_shell(string):
    
    list_symbol = [' ', '(', ')']
    list_replace = [r'\ ', r'\(', r'\)']
    
    for i, j in zip(list_symbol, list_replace):
        string = string.replace(i, j)
    return string

def rename_file_folder(path_folder, base_name, new_format):
# переименовать все файлы в папке 

    black_name = ''
    for counter, i in enumerate(os.listdir(path_folder)):
        if i != black_name:
            
            new_path = change_str_to_shell(os.path.join(path_folder, base_name) + str(counter) + new_format)
            
            subprocess.call('mv ' + change_str_to_shell(os.path.join(path_folder, i)) + ' ' + new_path, shell = True)
            #print('mv ' + os.path.join(path_folder, i).replace(' ', '\ ') + ' ' + new_path.replace(' ', '\ '))
            
        #fl = open(os.path.join(path_folder, i), 'r')
            #txt = fl.read()
            #fl.close()

            #fl = open(new_path, 'w')
            #fl.write(txt)
            #fl.close()
        
def create_description_file(size, number, name_file, name_folder, counter = 1):
    # создание файла описания 
    write_str = name_folder if name_folder[-1] == '/' else name_folder + '/'
    
    name_format = '.bmp'
    
    ret_str = ''
    
    const_list = [counter, 0, 0, size, size]
    const_str = ''
    black_name = '.DS_Store'
    
    for i in const_list:
        const_str += str(i) + ' '
    const_str = const_str[:-1]
    
    for i in range(number):
        if i != black_name:
            ret_str += write_str + str(i) + name_format + '  ' + const_str + '\n'
    
    fl = open(name_file, 'w')
    fl.write(ret_str[:-1])
    fl.close()

def create_bad_file(name_file, name_folder):
    name_format = '.bmp'
    black_name = '.DS_Store'
    
    path_to_folder = os.getcwd()
    path_save = os.path.join(path_to_folder, name_folder)
    
    ret_str = ''

    for i in os.listdir(path_save):
        if i != black_name:
            ret_str += os.path.join(path_save, i) + '\n'
    
    fl = open(name_file, 'w')
    fl.write(ret_str[:-1])
    fl.close()

In [6]:
Test = True

format_file = '.bmp'

number_thread = 6

#Time_start = time.time()
# зашумление исходного файла

image_noise, list_image = change_image(pix, size, last_size, 'test' + format_file, color_delete = 85)


#Time_1 = time.time()
#print(Time_1 - Time_start)
# создание пустого файла
image_full_noise = create_noise_image(image = image_noise, path_save = 'noise' + format_file)
#print(time.time() - Time_1)

path: test.bmp
1.5949852466583252
0.739659309387207


In [26]:
path_folder = 'Good'

for i in os.listdir(path_folder):
    if i != black_name:
        path_now = os.path.join(path_folder, i)
        image_now = Image.open(path_now)
        #pix = image_now.load()
        image_noise, list_image = change_image(image_now.load(), max(image_now.size), image_now.size,
                                               path = path_now, color_delete = 85)


In [1]:
ls

 AlgoritmRabochii.ipynb   Good_desc.dat            Rotate_Image.ipynb
 Bad/                     haarcascade/             samples.vec
 Bad_desc.dat             Image/                   vou.png
 from11.vec               new/                    'обученные каскады'/
 from2.vec                o11.vec
 Good/                    RazkadrovkaVideo.ipynb


In [12]:
sizepix_new = image_noise.load()
error_phi = 5
number_image = 400
# создание (error_phi * 2 / step) картинок с поворотом
list_phi = np.linspace(0, error_phi, number_image)

Time_now = time.time()
for counter, i in enumerate(list_phi):
    path = path_save + str(counter) + format_file
    rotate_image(image_noise, image_full_noise, pix_new, list_image, path, i)
print(Time_now - time.time())

-63.46161866188049


In [13]:
create_description_file(image_noise.size[0], len(list_phi), name_file = Name_Good, name_folder = Folder_Good)
create_bad_file(name_file = Name_Bad, name_folder = Folder_Bad)

In [14]:
rename_file_folder(Folder_Bad, '', '.bmp')

In [1]:

def videoStreamer(path, path_save, width=240, height=240, skip=None, name = '', format = '.bmp', number_image = 147, flag_image = True):
    # Загружаем видео.
    stream = cv2.VideoCapture(path)
    #print(len(stream))

    success, image = stream.read()
    
    #frames = int(stream.get(cv2.CAP_PROP_FRAME_COUNT))
    #FPS = stream.get(cv2.CAP_PROP_FPS)
    #if skip == None:
    #    skip = int(40)
    if flag_image:
        print(stream.get(7))
    else:
        name_dir = path_save.split('/')[-1]
        subprocess.call('rm -rf ' + change_str_to_shell(path_save), shell = True)
        subprocess.call('mkdir ' + change_str_to_shell(path_save), shell = True)
        print(name_dir)
        skip = skip if skip else (stream.get(7) / number_image)
        list_skip = list(map(int, np.linspace(0, stream.get(7), number_image)))
        
        print(stream.get(7), skip)
        counter = 0
        counter_image = 0
        print(path_save)
        while success:
            
          # Пропускаем несколько кадров, и смотрим один из них.
            #print(counter)
          # Меняем размер изображения и отсылаем его в stdout в формате JSON
            #frame = imutils.resize(frame, width=600, height=800)
            success, image = stream.read()
            if (counter) in list_skip and success:
                #print(list_skip[list_skip.index(counter)], counter)
                image = imutils.resize(image, width=width, height=height)
                cv2.imwrite(path_save + '/' + name +  str(counter_image) + format, image)
                #new_image = image.resize((width, height), Image.ANTIALIAS)
                #new_image.save(path_save + str(counter)+ '.png', optimize = True, quality = 95)
                counter_image += 1
            #print(counter)
            counter += 1
        print(counter_image)

In [14]:
create_description_file(40, 799, name_file = os.path.join(path_save, Name_Good), name_folder = Folder_Good)

In [7]:
path_save = '/home/adminivanov/Рабочий стол/Open_CV/Version_2'

In [51]:
__version__

'4.1.1'